In [ ]:
#seller_folios.xlsx final

import pandas as pd
from datetime import datetime


contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx')
seller_contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\seller_contacts.xlsx')


seller_df = contacts_df[
    (contacts_df['Roles'] == 'Seller') | 
    (contacts_df['Roles'] == 'Owner, Seller') | 
    (contacts_df['Roles'] == 'Seller, Supplier')
].copy()


seller_folios_data = []


def remove_dollar_sign(value):
    if pd.isna(value):
        return None
    return float(str(value).replace('$', '').replace(',', '').strip())


id_counter = 1000


for index, row in seller_df.iterrows():
    
    agreement_split = row['Inner_Agreement (Buyer)'].split(' to ') if pd.notna(row['Inner_Agreement (Buyer)']) else [None, None]
    agreement_start = agreement_split[0].replace('from ', '') if agreement_split[0] else None
    agreement_end = agreement_split[1] if len(agreement_split) > 1 else None
    
    
    agreement_start = pd.to_datetime(agreement_start).strftime('%Y-%m-%d') if agreement_start else None
    agreement_end = pd.to_datetime(agreement_end).strftime('%Y-%m-%d') if agreement_end else None

    
    asking_price = remove_dollar_sign(row['Inner_Asking Price'])
    commission = remove_dollar_sign(row['Inner_Commission'])
    folio_code = row['Inner_Folio Code (Buyer)'].split('(')[1].split(')')[0] if pd.notna(row['Inner_Folio Code (Buyer)']) else None
    money_in = remove_dollar_sign(row['Folio_MePay Pending'])
    money_out = remove_dollar_sign(row['Folio_Paid To'])
    balance = remove_dollar_sign(row['Folio_Deposits'])
    uncleared = remove_dollar_sign(row['Folio_Eff. Paid To'])

    
    seller_contact_row = seller_contacts_df[seller_contacts_df['reference'] == row['Reference']]
    seller_contact_id = seller_contact_row['id'].values[0] if not seller_contact_row.empty else None

    
    seller_folios_data.append({
        'id': id_counter,  # Auto incremental id starting from 1000
        'agreement_start': agreement_start,
        'agreement_end': agreement_end,
        'asking_price': asking_price,
        'commission': commission,
        'seller_contact_id': seller_contact_id,
        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'folio_code': folio_code,
        'company_id': 1,  # company_id is always 1
        'money_in': money_in,
        'money_out': money_out,
        'balance': balance,
        'uncleared': uncleared
    })

    
    id_counter += 1


seller_folios_df = pd.DataFrame(seller_folios_data)


seller_folios_df.to_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\seller_folios.xlsx', index=False)

print("seller_folios.xlsx created successfully!")
